In [1]:
# Importing and data
import theano.tensor as T
import theano
import sys, os
sys.path.append("../GeMpy")

# Importing GeMpy modules
import GeMpy

# Reloading (only for development purposes)
import importlib
importlib.reload(GeMpy)

# Usuful packages
import numpy as np
import pandas as pn

import matplotlib.pyplot as plt

# This was to choose the gpu
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Default options of printin
np.set_printoptions(precision = 6, linewidth= 130, suppress =  True)

#%matplotlib inline
%matplotlib inline



# Setting the extent
geo_data = GeMpy.import_data([0,10,0,10,0,10], [50,50,50])


# =========================
# DATA GENERATION IN PYTHON
# =========================
# Layers coordinates
layer_1 = np.array([[0.5,4,7], [2,4,6.5], [4,4,7], [5,4,6]])#-np.array([5,5,4]))/8+0.5
layer_2 = np.array([[3,4,5], [6,4,4],[8,4,4], [7,4,3], [1,4,6]])
layers = np.asarray([layer_1,layer_2])

# Foliations coordinates
dip_pos_1 = np.array([7,4,7])#- np.array([5,5,4]))/8+0.5
dip_pos_2 = np.array([2.,4,4])

# Dips
dip_angle_1 = float(15)
dip_angle_2 = float(340)
dips_angles = np.asarray([dip_angle_1, dip_angle_2], dtype="float64")

# Azimuths
azimuths = np.asarray([90,90], dtype="float64")

# Polarity
polarity = np.asarray([1,1], dtype="float64")

# Setting foliations and interfaces values
GeMpy.set_interfaces(geo_data, pn.DataFrame(
    data = {"X" :np.append(layer_1[:, 0],layer_2[:,0]),
            "Y" :np.append(layer_1[:, 1],layer_2[:,1]),
            "Z" :np.append(layer_1[:, 2],layer_2[:,2]),
            "formation" : np.append(
               np.tile("Layer 1", len(layer_1)), 
               np.tile("Layer 2", len(layer_2))),
            "labels" : [r'${\bf{x}}_{\alpha \, 0}^1$',
               r'${\bf{x}}_{\alpha \, 1}^1$',
               r'${\bf{x}}_{\alpha \, 2}^1$',
               r'${\bf{x}}_{\alpha \, 3}^1$',
               r'${\bf{x}}_{\alpha \, 0}^2$',
               r'${\bf{x}}_{\alpha \, 1}^2$',
               r'${\bf{x}}_{\alpha \, 2}^2$',
               r'${\bf{x}}_{\alpha \, 3}^2$',
        
                        r'${\bf{x}}_{\alpha \, 4}^2$'] }))

GeMpy.set_foliations(geo_data,  pn.DataFrame(
    data = {"X" :np.append(dip_pos_1[0],dip_pos_2[0]),
            "Y" :np.append(dip_pos_1[ 1],dip_pos_2[1]),
            "Z" :np.append(dip_pos_1[ 2],dip_pos_2[2]),
            "azimuth" : azimuths,
            "dip" : dips_angles,
            "polarity" : polarity,
            "formation" : ["Layer 1", "Layer 2"],
            "labels" : [r'${\bf{x}}_{\beta \,{0}}$',
              r'${\bf{x}}_{\beta \,{1}}$'] })) 

In [2]:
import vtk


In [1]:

import vtk
import random
 
 
'''
There are two alternative ways to apply the transform.
 1) Use vtkTransformPolyDataFilter to create a new transformed polydata. 
    This method is useful if the transformed polydata is needed
      later in the pipeline
    To do this, set USER_MATRIX = True
 2) Apply the transform directly to the actor using vtkProp3D's SetUserMatrix. 
    No new data is produced.
    To do this, set USER_MATRIX = False
'''
USER_MATRIX = True
 
#Create an arrow.
arrowSource = vtk.vtkArrowSource()
 
# Generate a random start and end point
random.seed(8775070)
startPoint = [0 for i in range(3)]
startPoint[0] = random.uniform(-10,10)
startPoint[1] = random.uniform(-10,10)
startPoint[2] = random.uniform(-10,10)
endPoint = [0 for i in range(3)]
endPoint[0] = random.uniform(-10,10)
endPoint[1] = random.uniform(-10,10)
endPoint[2] = random.uniform(-10,10)
 
# Compute a basis
normalizedX = [0 for i in range(3)]
normalizedY = [0 for i in range(3)]
normalizedZ = [0 for i in range(3)]
 
# The X axis is a vector from start to end
math = vtk.vtkMath()
math.Subtract(endPoint, startPoint, normalizedX)
length = math.Norm(normalizedX)
math.Normalize(normalizedX)
 
# The Z axis is an arbitrary vector cross X
arbitrary = [0 for i in range(3)]
arbitrary[0] = random.uniform(-10,10)
arbitrary[1] = random.uniform(-10,10)
arbitrary[2] = random.uniform(-10,10)
math.Cross(normalizedX, arbitrary, normalizedZ)
math.Normalize(normalizedZ)
 
# The Y axis is Z cross X
math.Cross(normalizedZ, normalizedX, normalizedY)
matrix = vtk.vtkMatrix4x4()
 
# Create the direction cosine matrix
matrix.Identity()
for i in range(3):
  matrix.SetElement(i, 0, normalizedX[i])
  matrix.SetElement(i, 1, normalizedY[i])
  matrix.SetElement(i, 2, normalizedZ[i])
 
# Apply the transforms
transform = vtk.vtkTransform()
transform.Translate(startPoint)
transform.Concatenate(matrix)
transform.Scale(length, length, length)
 
# Transform the polydata
transformPD = vtk.vtkTransformPolyDataFilter()
transformPD.SetTransform(transform)
transformPD.SetInputConnection(arrowSource.GetOutputPort())
 
#Create a mapper and actor for the arrow
mapper = vtk.vtkPolyDataMapper()
actor = vtk.vtkActor()
 
if USER_MATRIX:
    mapper.SetInputConnection(arrowSource.GetOutputPort())
    actor.SetUserMatrix(transform.GetMatrix())
else:
    mapper.SetInputConnection(transformPD.GetOutputPort())
 
actor.SetMapper(mapper)




 
# Create spheres for start and end point
sphereStartSource = vtk.vtkSphereWidget()
# sphereStartSource.SetCenter(startPoint)
# sphereStartMapper = vtk.vtkPolyDataMapper()
# sphereStartMapper.SetInputConnection(sphereStartSource.GetOutputPort())
# sphereStart = vtk.vtkActor()
# sphereStart.SetMapper(sphereStartMapper)
# sphereStart.GetProperty().SetColor(1.0, 1.0, .3)



sphereEndSource = vtk.vtkSphereSource()
sphereEndSource.SetCenter(endPoint)
sphereEndMapper = vtk.vtkPolyDataMapper()
sphereEndMapper.SetInputConnection(sphereEndSource.GetOutputPort())
sphereEnd = vtk.vtkActor()
sphereEnd.SetMapper(sphereEndMapper)
sphereEnd.GetProperty().SetColor(1.0, .3, .3)
 
#Create a renderer, render window, and interactor
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
    
sphereStartSource.SetInteractor(renderWindowInteractor)
sphereStartSource.SetRepresentationToSurface()
sphereStartSource.On()

    
    
    
# #Add the actor to the scene
# renderer.AddActor(actor)
# renderer.AddActor(sphereStart)
# renderer.AddActor(sphereEnd)
# renderer.SetBackground(.1, .2, .3) # Background color dark blue
 
 
#Render and interact
renderWindow.Render()
renderWindowInteractor.Start()

del renderWindow, renderWindowInteractor


In [2]:
 # First we import the VTK Python package that will make available all
 # of the VTK commands to Python.
 import vtk
 
# Next we create an instance of vtkConeSource and set some of its
# properties. The instance of vtkConeSource "cone" is part of a
# visualization pipeline (it is a source process object); it produces
# data (output type is vtkPolyData) which other filters may process.
cone = vtk.vtkConeSource()
cone.SetHeight(3.0)
cone.SetRadius(1.0)
cone.SetResolution(10)

# In this example we terminate the pipeline with a mapper process object.
# (Intermediate filters such as vtkShrinkPolyData could be inserted in
# between the source and the mapper.)  We create an instance of
# vtkPolyDataMapper to map the polygonal data into graphics primitives. We
# connect the output of the cone souece to the input of this mapper.
coneMapper = vtk.vtkPolyDataMapper()
coneMapper.SetInputConnection(cone.GetOutputPort())

# Create an actor to represent the cone. The actor orchestrates rendering of
# the mapper's graphics primitives. An actor also refers to properties via a
# vtkProperty instance, and includes an internal transformation matrix. We
# set this actor's mapper to be coneMapper which we created above.
coneActor = vtk.vtkActor()
coneActor.SetMapper(coneMapper)

# Create the Renderer and assign actors to it. A renderer is like a
# viewport. It is part or all of a window on the screen and it is
# responsible for drawing the actors it has.  We also set the
# background color here.
ren1 = vtk.vtkRenderer()
ren1.AddActor(coneActor)
ren1.SetBackground(0.1, 0.2, 0.4)

# Finally we create the render window which will show up on the screen
# We put our renderer into the render window using AddRenderer. We
# also set the size to be 300 pixels by 300.
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
renWin.SetSize(300, 300)
 
 # The vtkRenderWindowInteractor class watches for events (e.g., keypress,
 # mouse) in the vtkRenderWindow. These events are translated into
 # event invocations that VTK understands (see VTK/Common/vtkCommand.h
 # for all events that VTK processes). Then observers of these VTK
# events can process them as appropriate.
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# By default the vtkRenderWindowInteractor instantiates an instance
# of vtkInteractorStyle. vtkInteractorStyle translates a set of events
# it observes into operations on the camera, actors, and/or properties
# in the vtkRenderWindow associated with the vtkRenderWinodwInteractor.
# Here we specify a particular interactor style.
style = vtk.vtkInteractorStyleTrackballCamera()
iren.SetInteractorStyle(style)


# Here we use a vtkBoxWidget to transform the underlying coneActor (by
# manipulating its transformation matrix). Many other types of widgets
# are available for use, see the documentation for more details.
#
# The SetInteractor method is how 3D widgets are associated with the render
# window interactor. Internally, SetInteractor sets up a bunch of callbacks
# using the Command/Observer mechanism (AddObserver()). The place factor
# controls the initial size of the widget with respect to the bounding box
# of the input to the widget.
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(iren)
boxWidget.SetPlaceFactor(1.25)

# Place the interactor initially. The input to a 3D widget is used to
# initially position and scale the widget. The EndInteractionEvent is
# observed which invokes the SelectPolygons callback.
boxWidget.SetProp3D(coneActor)
boxWidget.PlaceWidget()

# Similar to Step2/Python/Cone2.py, we define a callback for
# interaction.  As can be seen the callback takes two arguments.  The
# first being the object that generates the event and the second
# argument the event name (which is a string).
def myCallback(widget, event_string):
    t = vtk.vtkTransform()
    boxWidget.GetTransform(t)
    boxWidget.GetProp3D().SetUserTransform(t)


# Now for every interaction event that is generated by the boxWidget,
# call our callback function.
boxWidget.AddObserver("InteractionEvent", myCallback)

# Normally the user presses the "i" key to bring a 3D widget to
# life. Here we will manually enable it so it appears with the cone.
boxWidget.On()

# Start the event loop.
iren.Initialize()
iren.Start()

del renwin, iren
# There is no explicit need to free any objects at this point.

NameError: name 'renwin' is not defined

In [7]:
renderer.SetBackground?

In [7]:
import vtk
 
# Call back function
def sphereCallback(obj, event):
    camera.SetFocalPoint(5, 5, 5);
    camera.SetPosition(0,5,5)
    camera.SetViewUp(0,0,1)
    renderer.ResetCamera()
    print('Center: {}, {}, {}'.format(*obj.GetCenter()))
 
# A renderer and render window
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
#renderer.SetBackground(0, 0, 1)
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
renwin.SetSize(700, 700)
renwin.SetWindowName('RW: Multiple ViewPorts')
#renderer.SetViewport(0.8,0,1,1)

# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)




# A Sphere widget
sphereWidget = vtk.vtkSphereWidget()
sphereWidget.SetCenter(5,5,1)
sphereWidget.SetInteractor(interactor)
sphereWidget.SetRepresentationToSurface()
sphereWidget.On()

sphereWidget2 = vtk.vtkSphereWidget()
sphereWidget2.SetInteractor(interactor)
sphereWidget2.SetRepresentationToSurface()
sphereWidget2.On()
 
# Connect the event to a function
sphereWidget.AddObserver("InteractionEvent", sphereCallback)
sphereWidget2.AddObserver("InteractionEvent", sphereCallback)
 
    
    


cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
#cubeAxesActor.SetAxisBaseForZ(0,1,0)
#cubeAxesActor.SetAxisBaseForY(0,0,1)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(1.0, 1.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(1.0, 1.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
# cubeAxesActor.XAxisMinorTickVisibilityOff()
# cubeAxesActor.YAxisMinorTickVisibilityOff()
# cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)

camera =vtk.vtkCamera ();
camera.SetFocalPoint(5, 5, 5);
camera.SetPosition(0,5,5)
camera.SetViewUp(0,0,1)


renderer.SetActiveCamera(camera);

renderer.ResetCamera()
cubeAxesActor.SetCamera(renderer.GetActiveCamera())


# AXES
transform = vtk.vtkTransform()
transform.Translate(1.0, 0.0, 0.0)

 
# properties of the axes labels can be set as follows
# this sets the x axis label to red
# axes->GetXAxisCaptionActor2D()->GetCaptionTextProperty()->SetColor(1,0,0);
 
# the actual text of the axis label can be changed:
# axes->SetXAxisLabelText("test");
 
#renderer.AddActor(axes)

arrowSource = vtk.vtkArrowSource()
#arrowSource.SetShaftRadius(0.01)
#arrowSource.SetTipLength(.9)
 
# Create a mapper and actor
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(arrowSource.GetOutputPort())
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.AddObserver("InteractionEvent", sphereCallback)
renderer.AddActor(actor)

# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

In [87]:
axes.SetPosition(-5, -5, - 5)

In [25]:
camera =vtk.vtkCamera ();
camera.

In [37]:
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetEventInformation()

TypeError: no overloads of SetEventInformation() take 0 arguments

In [23]:
renderer = vtk.vtkRenderer()
# renderer.SetBackground(10, 10, 10)
 
renderer.SetViewport?

### Box limit?

In [67]:
import vtk
 
# Call back function to resize the cone
def boxCallback(obj, event):
    t = vtk.vtkTransform()
    obj.GetTransform(t)
    obj.GetProp3D().SetUserTransform( t )

    
    
    
    
# A renderer and render window
renderer = vtk.vtkRenderer()
renderer.SetBackground(0, 0, 1)
# renderer.AddActor(coneActor)
 
renwin = vtk.vtkRenderWindow()
renwin.AddRenderer(renderer)
 
# An interactor
interactor = vtk.vtkRenderWindowInteractor()
interactor.SetRenderWindow(renwin)



cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(0, 10, 0, 10, 0, 10)
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
#cubeAxesActor.DrawXGridlinesOn()
#cubeAxesActor.DrawYGridlinesOn()
#cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()

  
renderer.AddActor(cubeAxesActor)

# A Box widget
boxWidget = vtk.vtkBoxWidget()
boxWidget.SetInteractor(interactor)
boxWidget.SetProp3D(cubeAxesActor)
boxWidget.SetPlaceFactor( 1.25 ) # Make the box 1.25x larger than the actor
boxWidget.PlaceWidget()
boxWidget.On()
 
# Connect the event to a function
boxWidget.AddObserver("InteractionEvent", boxCallback)
 
# Start
interactor.Initialize()
interactor.Start()

del renwin, interactor

In [58]:
cubeAxesActor.GetBounds()

(-1.0, 1.0, -1.0, 1.0, -1.0, 1.0)

In [40]:
#!/usr/bin/env python
 
import vtk
 
# Create a superquadric
superquadricSource = vtk.vtkSuperquadricSource()
superquadricSource.SetPhiRoundness(3.1)
superquadricSource.SetThetaRoundness(1.0)
superquadricSource.Update()  # needed to GetBounds later
 
renderer = vtk.vtkRenderer()
 
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(superquadricSource.GetOutputPort())
 
superquadricActor = vtk.vtkActor()
superquadricActor.SetMapper(mapper)
 
cubeAxesActor = vtk.vtkCubeAxesActor()
cubeAxesActor.SetBounds(superquadricSource.GetOutput().GetBounds())
cubeAxesActor.SetCamera(renderer.GetActiveCamera())
cubeAxesActor.GetTitleTextProperty(0).SetColor(1.0, 0.0, 0.0)
cubeAxesActor.GetLabelTextProperty(0).SetColor(1.0, 0.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(1).SetColor(0.0, 1.0, 0.0)
cubeAxesActor.GetLabelTextProperty(1).SetColor(0.0, 1.0, 0.0)
 
cubeAxesActor.GetTitleTextProperty(2).SetColor(0.0, 0.0, 1.0)
cubeAxesActor.GetLabelTextProperty(2).SetColor(0.0, 0.0, 1.0)
 
cubeAxesActor.DrawXGridlinesOn()
cubeAxesActor.DrawYGridlinesOn()
cubeAxesActor.DrawZGridlinesOn()
if vtk.VTK_MAJOR_VERSION > 5:
    cubeAxesActor.SetGridLineLocation(vtk.VTK_GRID_LINES_FURTHEST)
 
cubeAxesActor.XAxisMinorTickVisibilityOff()
cubeAxesActor.YAxisMinorTickVisibilityOff()
cubeAxesActor.ZAxisMinorTickVisibilityOff()
 
renderer.AddActor(cubeAxesActor)
renderer.AddActor(superquadricActor)
renderer.GetActiveCamera().Azimuth(30)
renderer.GetActiveCamera().Elevation(30)
 
renderer.ResetCamera()
 
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
 
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
renderWindow.Render()
renderWindowInteractor.Start()

del renderWindow, renderWindowInteractor

In [31]:
cubeAxesActor.SetBounds?